In [84]:
%load_ext autoreload
%autoreload 2

In [63]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time
import json
import os
import requests
import re
import pickle
from nltk import ngrams

### Notebook for finding and cleaning Business names and entities.

#### Logic:

* Check the 'Address Book Type'
    * If Business Enttity: Run cleaning logic on "Name" field.
    * If Individual/Candidate Family: Run cleaning logic on "Employer Name" field.
* Create a dictionary of 

In [4]:
transaction_detail_df = pd.read_csv("../scrape_files/transaction_detail_first_batch.csv", low_memory=False)

In [71]:
with open("state_city.pickle", 'rb') as f:
    state_city_dict = pickle.load(f)

In [28]:
for col in transaction_detail_df.columns:
    print(col)

Address
Address Book Type
Agent
Aggregate
Amount
Associations
Check
CoSigner Obligations
Description
Due Date
Employer Name
Exam Letter Date
Filed Date
In-Kind/Independent Expenditures
Interest Rate
Name
Occupation
Occupation Letter Date
Payer of Personal Expenditure
Payment Method
Process Status
Purpose
Repayment Schedule
Transaction Date
Transaction ID
Transaction Sub Type
Transaction Type


In [5]:
np.unique(list(transaction_detail_df["Address Book Type"]))

array(['Business Entity', "Candidate's Immediate Family", 'Individual',
       'Labor Organization', 'Other', 'Political Committee',
       'Political Party Committee', 'Unregistered Committee', 'nan'],
      dtype='<U28')

In [57]:
business_sub_df = transaction_detail_df[transaction_detail_df["Address Book Type"] == "Business Entity"].copy()
individual_sub_df = transaction_detail_df[transaction_detail_df["Address Book Type"] == "Individual"].copy()

In [ ]:
def find_matching_city(state, city):
    

In [60]:
for i in np.unique(list(individual_sub_df["Employer Name"])):
    i = i.lower()
    i = i.split()

['services"', 'salem', 'or']
"STATE OF OREGON, DEPARTMENT OF HUMAN SERVICES" SALEM OR
['oregon', 'portland', 'or']
100 Friends of Oregon Portland OR
['new', 'york', 'ny']
1199 SEIU Benefit and Pension Funds New York NY
['llc', 'aloha', 'or']
2030 Investors LLC Aloha OR
['research', 'potland', 'or']
22 Research Potland OR
['llp', 'portland', 'or']
29F Architects LLP Portland OR
['architecture', 'eugene', 'or']
2form Architecture Eugene OR
['consulting', 'portland', 'or']
3J Consulting Portland OR
['district', 'eugene', 'or']
4 J school district eugene OR
['district', 'eugene', 'or']
4J School District Eugene OR
['market', 'eugene', 'or']
5th Street Public Market and Inn at the Market Eugene OR
['dist', 'ontario', 'or']
8-C School Dist Ontario OR
['dist', 'ontario', 'or']
8-C Schoold Dist Ontario OR
['inc.', 'portland', 'or']
88, Inc. Portland OR
['taqueria', 'portland', 'or']
?Por Qu? No? Taqueria Portland OR
['films', 'portland', 'or']
@ Large Films Portland OR
['films', 'portland', 'o

In [21]:
for i in np.unique(list(individual_sub_df["Employer Name"])):
    if len(i) < 15:
        print(i)

Brian clem
COPA Bend OR
City of Eugene
EWEB Eugene OR
EY Portland OR
HYA Chicago IL
MA Seattle WA
Not Employed
OSBA Salem OR
OTBC Durham OR
RootX Salem OR
Self Employed
Self Salem OR
USI Eugene OR
nan


In [7]:
len(np.unique(list(business_sub_df["Name"])))/len(business_sub_df)

0.2108880007798031

Create a dict of arrays: original_name: [[% match, name], ...]

In [ ]:
len(np.unique(list(business_sub_df["Name"])))

In [ ]:
np.unique(list(business_sub_df["Name"]))[10]

In [ ]:
query = np.unique(list(business_sub_df["Name"]))[10]


In [76]:
# Get a list of matches ordered by score, default limit to 5
choices = np.unique(list(business_sub_df["Name"]))

In [77]:
top_choices_dict = {}
start_time = time.time()
full_list = list(np.unique(list(business_sub_df["Name"])))

In [107]:
query = full_list[0]

In [100]:
from fuzzymatchlist import FuzzyList

In [104]:
FL = FuzzyList(choices)

In [109]:
query

'1 & 1 Interent'

In [112]:
import string

In [126]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [129]:
translator= str.maketrans("", "", string.punctuation)

In [130]:
query.translate(translator)

'1  1 Interent'

In [125]:
"$&@*@".strip()

'$&@*@'

In [124]:
type(apunc)

str

In [110]:
FL.get_top_n_matches(query.strip())

[('1 and 1 Internet, Inc.', 86),
 ('LivWell Oregon 1, LLC', 86),
 ('1&1 Internet Inc', 75),
 ('1&1 Internet Inc.', 75),
 ('Intel', 72),
 ('American Pacific International Capital Inc.', 68),
 ('Aristotle International, Inc.', 68),
 ('Dai Ichi International Travel', 68),
 ('Heery International Inc', 68),
 ('Heery International, Inc', 68),
 ('Heery International, Inc.', 68),
 ('International Union of Painters & Allied Trades Political Co', 68),
 ('Marcois Partners International Corporati', 68),
 ('Stratus International Incorporated', 68),
 ('Teen Challenge International - PNC - Williamette Valley TC', 68),
 ('Teen Challenge International - Pacific Northwest Centers -', 68),
 ('111 Investments', 64),
 ('Amy McLeod Group, REMAX Integrity Agency', 60),
 ('Blueprint Interactive', 60),
 ('Bo & Vine', 60)]

In [ ]:
top_choices_dict = {}
start_time = time.time()
full_list = list(np.unique(list(business_sub_df["Name"])))
for i,name in enumerate(full_list):
    query = name
    choices = full_list[:i] + full_list[i+1:]
    top_choices_dict[name] = process.extract(query, choices, limit=20)
    if i % 100 == 0:
        print(str(np.round(i/len(full_list),3) * 100) + "%")
        print(str(time.time() - start_time))
        with open("match_dict.json", 'a') as f:
            json.dump(top_choices_dict, f, indent=4)
        print(os.stat("match_dict.json").st_size, " ", len(top_choices_dict))
        del(top_choices_dict)
        top_choices_dict = {}
print(str(np.round(i/len(full_list),3) * 100) + "%")
print(str(time.time() - start_time))
with open("match_dict.json", 'a') as f:
    json.dump(top_choices_dict, f, indent=4)

In [ ]:
for item in list(top_choices_dict.items())[:10]:
    print(item, '\n')

jccard index with ngrams

In [ ]:
with open("match_dict.json") as f:
    raw = f.read()

In [ ]:
"}{".split("}")

In [ ]:
splits = re.split("}{", raw)

In [ ]:
len(splits)

In [ ]:
json.loads(splits[0] + "}")